# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [75]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [76]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [77]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [78]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [116]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10,verbose=False)

Find what features had non-zero weight.

In [117]:
model_not_zeor = model_all['coefficients']['value'].nnz()

In [118]:
print model_not_zeor

6


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [82]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [120]:
import numpy as np
Rss = 1e20
num = -1
tmprss = 0
ls = dict()
for l1_penalty in np.logspace(1, 7, num=13):
    tmp_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
    tmprss = pow((tmp_model.predict(validation)-validation['price']),2).sum()
    ls[l1_penalty] = tmprss


In [121]:
mis = 1e20
nus = 0
print ls
for item in ls:
    if ls[item] < mis:
        mis = ls[item]
        nus = item
print nus

{100000.0: 625767302791634.1, 316227.76601683791: 625769507643886.2, 100.0: 625766286057885.0, 3162.2776601683795: 625766317206080.5, 31.622776601683793: 625766285362394.1, 1000.0: 625766295212186.8, 10.0: 625766285142459.9, 31622.776601683792: 625766606749278.5, 10000.0: 625766386760658.1, 3162277.6601683795: 625799062845467.0, 316.22776601683796: 625766288257224.6, 1000000.0: 625776517727024.0, 10000000.0: 625883719085425.2}
10.0


In [122]:

model_best = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10.0)


Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.952963     | 5757083.459662     | 280315.641755 |

| 6         | 7        | 0.000002  | 1.024818     | 5678327.292256     | 276610.230084 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [123]:
model_test_zeor = model_best['coefficients']['value'].nnz()
print model_test_zeor

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [100]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [101]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [130]:
ls =  dict()
for l1_penaltys in np.logspace(8, 10, num=20):
    stmp_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penaltys,verbose=False)
    ls[l1_penaltys] = stmp_model['coefficients']['value'].nnz()
print ls

{100000000.0: 18, 206913808.11147901: 18, 1128837891.6846883: 15, 6158482110.6602545: 3, 1832980710.8324375: 13, 3792690190.7322536: 6, 2976351441.6313133: 10, 127427498.57031322: 18, 545559478.11685145: 17, 1438449888.2876658: 15, 263665089.87303555: 17, 162377673.91887242: 18, 885866790.41008317: 16, 10000000000.0: 1, 335981828.62837881: 17, 7847599703.5146227: 1, 695192796.17755914: 17, 4832930238.5717525: 5, 2335721469.0901213: 12, 428133239.8719396: 17}


In [131]:
for key in sorted(ls.iterkeys()):
    print key
    print ls[key]

100000000.0
18
127427498.57
18
162377673.919
18
206913808.111
18
263665089.873
17
335981828.628
17
428133239.872
17
545559478.117
17
695192796.178
17
885866790.41
16
1128837891.68
15
1438449888.29
15
1832980710.83
13
2335721469.09
12
2976351441.63
10
3792690190.73
6
4832930238.57
5
6158482110.66
3
7847599703.51
1
10000000000.0
1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [142]:
l1_penalty_max = 3792000000
l1_penalty_min = 2976000000

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [143]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [148]:
ls2 = dict()
ls3 = dict()
for l1_penalty in l1_penalty_values:
    tmp_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
    tmprss = tmp_model.evaluate(validation)['rmse']
    tmprss = pow((tmp_model.predict(validation)-validation['price']),2).sum()
    ls2[l1_penalty] = tmprss
    ls3[l1_penalty] = tmp_model['coefficients']['value'].nnz()
print ls2
print ls3

{2976000000.0: 966867972181711.1, 3319578947.3684211: 1034571768283331.2, 3663157894.7368422: 1069408542489103.5, 3190736842.1052632: 1008377115058031.4, 3534315789.4736843: 1055926745402264.4, 3061894736.8421054: 981123450661093.8, 3405473684.2105265: 1043177114549521.2, 3749052631.5789475: 1077568025552419.2, 3792000000.0: 1081796307766041.6, 3276631578.9473686: 1028137030421132.8, 3620210526.3157892: 1065653692792079.1, 3147789473.6842108: 998688484348069.0, 3491368421.0526314: 1051084109839678.0, 3018947368.4210525: 973958135465239.4, 3362526315.7894735: 1038507666466976.9, 3706105263.1578951: 1073441837407740.0, 3233684210.5263157: 1018193297082384.9, 3577263157.8947368: 1060732168371792.2, 3104842105.2631578: 989250379948665.8, 3448421052.6315789: 1046890259812444.0}
{2976000000.0: 10, 3319578947.3684211: 8, 3663157894.7368422: 6, 3190736842.1052632: 10, 3534315789.4736843: 7, 3061894736.8421054: 10, 3405473684.2105265: 8, 3749052631.5789475: 6, 3792000000.0: 6, 3276631578.947368

In [150]:
for key in sorted(ls3.iterkeys()):
    print key
    print ls3[key]

2976000000.0
10
3018947368.42
10
3061894736.84
10
3104842105.26
10
3147789473.68
10
3190736842.11
10
3233684210.53
10
3276631578.95
10
3319578947.37
8
3362526315.79
8
3405473684.21
8
3448421052.63
7
3491368421.05
7
3534315789.47
7
3577263157.89
7
3620210526.32
6
3663157894.74
6
3706105263.16
6
3749052631.58
6
3792000000.0
6


In [152]:
3448421052.6315789: 1046890259812444.0
3491368421.0526314: 1051084109839678.0
3534315789.4736843: 1055926745402264.4
3577263157.8947368: 1060732168371792.2

KeyError: '3448421052.63'

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [153]:
models = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448421052.6315789,verbose=False)
print models.coefficients

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222223.676046 |  None  |
|     bedrooms     |  None | 663.710099151 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15876.7757028 |  None  |
|   sqft_living    |  None | 32.4126849707 |  None  |
| sqft_living_sqrt |  None | 690.249965581 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
+------------------+-------+---------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [154]:
print models['coefficients'].print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222223.676046 |  None  |
|     bedrooms     |  None | 663.710099151 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15876.7757028 |  None  |
|   sqft_living    |  None | 32.4126849707 |  None  |
| sqft_living_sqrt |  None | 690.249965581 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None |  2900.3220908 |  None  |
|    sqft_above    |  None | 30.0152077267 |  None  |
|  sqft_basement   |  None |